In [1]:
from tavily import TavilyClient
import asyncio, os, requests, time, json
from IPython.display import display, Markdown, Latex

tavily_client = TavilyClient(api_key="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")

In [2]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://34.29.243.149:8877/v1",
    api_key="token-abc123",
)

In [3]:
def deduplicate_and_format_sources(search_response, max_tokens_per_source, include_raw_content=True):
     # Collect all results
    sources_list = []
    for response in search_response:
        sources_list.extend(response['results'])
    
    # Deduplicate by URL
    unique_sources = {source['url']: source for source in sources_list}

    # Format output
    formatted_text = "Content from sources:\n"
    for i, source in enumerate(unique_sources.values(), 1):
        formatted_text += f"{'='*80}\n"  # Clear section separator
        formatted_text += f"Source: {source['title']}\n"
        formatted_text += f"{'-'*80}\n"  # Subsection separator
        formatted_text += f"URL: {source['url']}\n===\n"
        formatted_text += f"Most relevant content from source: {source['content']}\n===\n"
        if include_raw_content:
            # Using rough estimate of 4 characters per token
            char_limit = max_tokens_per_source * 4
            # Handle None raw_content
            raw_content = source.get('raw_content', '')
            if raw_content is None:
                raw_content = ''
                print(f"Warning: No raw_content found for source {source['url']}")
            if len(raw_content) > char_limit:
                raw_content = raw_content[:char_limit] + "... [truncated]"
            formatted_text += f"Full source content limited to {max_tokens_per_source} tokens: {raw_content}\n\n"
        formatted_text += f"{'='*80}\n\n" # End section separator
                
    return formatted_text.strip()

In [4]:
def generate_response(message_list):
    completion = client.chat.completions.create(
        model = "Llama-3.1-8B-Instruct",
        messages = message_list,
        max_tokens=2048,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )
    
    final_answer = []
    assistant_response = ""
    
    start = time.time()
    
    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content
        
        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content
    
    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response

In [5]:
import threading

def worker(query, search_result, req_num_result, include_raw, req_topic):
    print(f"Thread: {query}")
    search_result.append(
        tavily_client.search(
            query,
            max_results= req_num_result,
            include_raw_content= include_raw,
            topic= req_topic
        )
    )

In [6]:
def ask_tavily(search_queries, search_tasks, req_num_result, include_raw, req_topic):    
    start_time = time.time()
    threads = []
    
    for query in search_queries:
        t = threading.Thread(target=worker, args=(query, search_tasks, req_num_result, include_raw, req_topic))
        threads.append(t)
        t.start()

    for thread in threads:
        thread.join()
    
    end_time = time.time()
    execution_time = end_time - start_time

    print(f"\nask_tavily task running time: {execution_time:.2f}초 \n")

In [7]:
def ask_plan_query_writer(topic, content):
    llm_prompt = """You are an expert technical writer crafting a section that synthesizes information 
<section topic>
""" + topic + """
</section topic>

<section organization>
""" + content + """
</section organization>

<Task>
Your goal is to generate 3 web search queries that will help gather information for planning the sections. 

The queries should:

1. Be related to the section topic
2. Help satisfy the requirements specified in the section organization

Make the queries specific enough to find high-quality, relevant sources while covering the breadth needed for the section structure.

Note1. that today's date is """+time.strftime("%Y-%m-%d")+""".
Note2. Output your response in JSON format, with the following structure: { "queries": [ "query1", "query2", "query3" ] }
</Task>"""

    return llm_prompt

In [8]:
def ask_final_writer_instructions(topic, content, search_tasks):
    final_section_writer="""You are an expert technical writer.

<Section name>
""" + content + """
</Section name>

<Section topic> 
""" + topic + """
</Section topic>

<Available Website Search Content>
""" + deduplicate_and_format_sources(search_tasks, max_tokens_per_source=4000, include_raw_content=True) + """
</Available Website Search Content>

<Task>
1. Section-Specific Approach:

For Introduction:
- Use # for Website Search title (Markdown format)
- Write in simple and clear language
- Focus on the core motivation for the Section in 1-2 paragraphs
- Use a clear narrative arc to introduce the Section
- Include NO structural elements (no lists or tables)
- No sources section needed

For Conclusion/Summary:
- Use ## for Conclusion/Summary title (Markdown format)
- For comparative Conclusion/Summary:
    * Must include a focused comparison table using Markdown table syntax
    * Table should distill insights from the Section
    * Keep table entries clear and concise
- For non-comparative Conclusion/Summary: 
    * Only use ONE structural element IF it helps distill the points made in the Section:
    * Either a focused table comparing items present in the Section (using Markdown table syntax)
    * Or a short list using proper Markdown list syntax:
      - Use `*` or `-` for unordered lists
      - Use `1.` for ordered lists
      - Ensure proper indentation and spacing
- Sources and url section needed. (especially when expressing a URL, please provide the entire URL exactly as given in the content without abbreviating it.)
- End with specific next steps or implications

2. Writing Approach:
- Use concrete details over general statements
- Make every word count
- Focus on your single most important point
</Task>

<Quality Checks>
- Verify that EVERY claim is grounded in the provided Source material
- Confirm each URL appears ONLY ONCE in the Source list
- For introduction: # for Website Search title, no structural elements, no sources section
- For conclusion: ## for Conclusion/Summary title, only ONE structural element at most, add sources and url section
- Markdown format
- Do not include word count or any preamble in your response
</Quality Checks>

Please note that respond in Korean always."""

    return final_section_writer

In [9]:
system_prompt = "You are a helpful assistant. And Answers must be in Korean."

topic = "기술동향"
content = "MCP(model context protocol) 과 A2A(Agent to Agent) 는 어떤 차이가 있는것인지 알려줘."

messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": ask_plan_query_writer(topic, content)},
    ]

response_query = generate_response(messages)

{ 
  "queries": [
    "MCP vs A2A 프로토콜: 주요 차이점",
    "2025년 현재 MCP 및 A2A 프로토콜의 최신 기술 동향",
    "MCP 및 A2A 프로토콜의 비교: 모델 컨텍스트 프로토콜과 에이전트 투 에이전트"
  ]
}

inference time: 2.72356 sec 




In [10]:
json_data = json.loads(response_query)
queries = json_data['queries']

print("사용자 발화 기반으로 추출한 web query 문장 3건:")
print(queries)

search_tasks = []
req_topic = 'general' # news   gerneral 과 news 중 선택
req_num_result = 3    # 각 web query 에 대해 리턴할 site 개수
include_raw = False    # site 의 원본 컨텐츠 리턴 유무

ask_tavily(queries, search_tasks, req_num_result, include_raw, req_topic)
print(search_tasks)

사용자 발화 기반으로 추출한 web query 문장 3건:
['MCP vs A2A 프로토콜: 주요 차이점', '2025년 현재 MCP 및 A2A 프로토콜의 최신 기술 동향', 'MCP 및 A2A 프로토콜의 비교: 모델 컨텍스트 프로토콜과 에이전트 투 에이전트']
Thread: MCP vs A2A 프로토콜: 주요 차이점
Thread: 2025년 현재 MCP 및 A2A 프로토콜의 최신 기술 동향
Thread: MCP 및 A2A 프로토콜의 비교: 모델 컨텍스트 프로토콜과 에이전트 투 에이전트

ask_tavily task running time: 2.25초 

[{'query': 'MCP vs A2A 프로토콜: 주요 차이점', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'A2A vs MCP: 새로운 에이전트 생태계를 위한 두 개의 보완적 프로토콜 · Logto 블로그', 'url': 'https://blog.logto.io/ko/a2a-mcp', 'content': '이 기사에서는 AI 에이전트 시스템의 미래를 형성하는 두 가지 프로토콜, A2A와 MCP를 소개합니다. 이들이 어떻게 작동하고, 어떻게 다른지, 그리고 이 아키텍처를 이해하는 것이 개발자, 디자이너 및 AI 제품 제작자에게 왜 중요한지를 설명합니다.', 'score': 0.7325349, 'raw_content': None}, {'title': '구글의 A2a, Ai 에이전트 시대의 새로운 표준이 될까? - Mcp와의 비교부터 실제 사용 예시까지 한눈에 정리', 'url': 'https://digitalbourgeois.tistory.com/1039', 'content': '구글의 A2A, AI 에이전트 시대의 새로운 표준이 될까? 구글의 A2A, AI 에이전트 시대의 새로운 표준이 될까? 🛠 실제 사용 예시 – A2A 에이전트 체험기 주 용도에이전트 간 협업LLM 기능 확장 및 도구 연결아키텍처클라이

In [11]:
print("\n\n=================================================================\n")
messages.append({"role": "user", "content": ask_final_writer_instructions(topic, content, search_tasks)})
response_query = generate_response(messages)

print("\n\n=========================  Search Report  ========================================\n")
display(Markdown(response_query))




<section topic>
기술동향
</section topic>

<section organization>
MCP(model context protocol) 과 A2A(Agent to Agent) 는 어떤 차이가 있는것인지 알려줘.
</section organization>

<Task>
1. Section-Specific Approach:

# MCP와 A2A의 차이점

MCP(model context protocol)과 A2A(Agent to Agent)는 AI 에이전트가 다양한 도구와 서비스와 원활하게 상호작용할 수 있도록 하는 표준화된 프로토콜입니다. 하지만 두 프로토콜은 서로 다른 특징과 목적을 가지고 있습니다. 이 섹션에서는 MCP와 A2A의 차이점을 비교하고 분석할 것입니다.

MCP는 AI 모델과 에이전트가 다양한 도구와 서비스와 원활하게 상호작용할 수 있도록 하는 표준화된 프로토콜입니다. MCP는 AI 애플리케이션의 자원 관리 및 최적화를 위한 도구입니다. 반면에 A2A는 에이전트 간 협업을 위한 표준화된 프로토콜입니다. A2A는 에이전트 간의 통신과 협력을 쉽게 할 수 있도록 하는 도구입니다.

두 프로토콜 모두 AI 에이전트가 다양한 도구와 서비스와 원활하게 상호작용할 수 있도록 하는 표준화된 프로토콜입니다. 하지만 MCP는 AI 애플리케이션의 자원 관리 및 최적화를 위한 도구로 사용되며, A2A는 에이전트 간 협업을 위한 표준화된 프로토콜로 사용됩니다.

## 결론

| 특징 | MCP | A2A |
| --- | --- | --- |
| 목적 | AI 애플리케이션의 자원 관리 및 최적화 | 에이전트 간 협업 |
| 기능 | 자원 관리 및 최적화 | 통신 및 협력 |

Sources:
- https://blog.logto.io/ko/a2a-mcp
- https://digitalbourgeois.tistory.com/1039
- https://nextitnow.tistory.com/entry/MCP-API-AI-Agent의-차이점-

<section topic>
기술동향
</section topic>

<section organization>
MCP(model context protocol) 과 A2A(Agent to Agent) 는 어떤 차이가 있는것인지 알려줘.
</section organization>

<Task>
1. Section-Specific Approach:

# MCP와 A2A의 차이점

MCP(model context protocol)과 A2A(Agent to Agent)는 AI 에이전트가 다양한 도구와 서비스와 원활하게 상호작용할 수 있도록 하는 표준화된 프로토콜입니다. 하지만 두 프로토콜은 서로 다른 특징과 목적을 가지고 있습니다. 이 섹션에서는 MCP와 A2A의 차이점을 비교하고 분석할 것입니다.

MCP는 AI 모델과 에이전트가 다양한 도구와 서비스와 원활하게 상호작용할 수 있도록 하는 표준화된 프로토콜입니다. MCP는 AI 애플리케이션의 자원 관리 및 최적화를 위한 도구입니다. 반면에 A2A는 에이전트 간 협업을 위한 표준화된 프로토콜입니다. A2A는 에이전트 간의 통신과 협력을 쉽게 할 수 있도록 하는 도구입니다.

두 프로토콜 모두 AI 에이전트가 다양한 도구와 서비스와 원활하게 상호작용할 수 있도록 하는 표준화된 프로토콜입니다. 하지만 MCP는 AI 애플리케이션의 자원 관리 및 최적화를 위한 도구로 사용되며, A2A는 에이전트 간 협업을 위한 표준화된 프로토콜로 사용됩니다.

## 결론

| 특징 | MCP | A2A |
| --- | --- | --- |
| 목적 | AI 애플리케이션의 자원 관리 및 최적화 | 에이전트 간 협업 |
| 기능 | 자원 관리 및 최적화 | 통신 및 협력 |

Sources:
- https://blog.logto.io/ko/a2a-mcp
- https://digitalbourgeois.tistory.com/1039
- https://nextitnow.tistory.com/entry/MCP-API-AI-Agent의-차이점-완전-정리
- https://botpress.com/ko/blog/model-context-protocol
- https://dma-ai.kr/77
- https://dma-ai.kr/81
- https://sugar-family.tistory.com/548

다음으로는 MCP와 A2A를 사용하는 경우에 대한 고려 사항을 살펴볼 것입니다.

## 다음 단계

MCP와 A2A를 사용하는 경우에 대한 고려 사항은 다음과 같습니다.

*   MCP를 사용하는 경우, AI 애플리케이션의 자원 관리 및 최적화를 위해 사용해야 합니다.
*   A2A를 사용하는 경우, 에이전트 간 협업을 위해 사용해야 합니다.
*   두 프로토콜 모두 AI 에이전트가 다양한 도구와 서비스와 원활하게 상호작용할 수 있도록 하는 표준화된 프로토콜입니다.

Sources:
- https://blog.logto.io/ko/a2a-mcp
- https://digitalbourgeois.tistory.com/1039
- https://nextitnow.tistory.com/entry/MCP-API-AI-Agent의-차이점-완전-정리
- https://botpress.com/ko/blog/model-context-protocol
- https://dma-ai.kr/77
- https://dma-ai.kr/81
- https://sugar-family.tistory.com/548

</Task>

<Quality Checks>
EVERY claim is grounded in the provided Source material.
각 URL은 Source list에서 ONLY 한 번만 나타납니다.
introduction: # for Website Search title, no structural elements, no sources section.
conclusion: ## for Conclusion/Summary title, only ONE structural element at most, add sources and url section.
Markdown format.
word count 또는 preamble 포함되지 않습니다.
</Quality Checks>

<Task>
2. Writing Approach:

Writing Approach를 위해 다음 내용을 추가합니다.

*   Concrete details over general statements: 
    +   "MCP는 AI 모델과 에이전트가 다양한 도구와 서비스와 원활하게 상호작용할 수 있도록 하는 표준화된 프로토콜입니다." 대신 "MCP는 AI 모델과 에이전트가 다양한 도구와 서비스를 연결하는 데 사용되는 표준화된 프로토콜입니다."
*   Make every word count:
    +   "두 프로토콜은 서로 다른 특징과 목적을 가지고 있습니다." 대신 "MCP와 A2A는 서로 다른 기능과 목적을 가지고 있습니다."
*   Focus on your single most important point:
    +   "두 프로토콜 모두 AI 에이전트가 다양한 도구와 서비스와 원활하게 상호작용할 수 있도록 하는 표준화된 프로토콜입니다." 대신 "MCP는 AI 모델과 에이전트가 다양한 도구와 서비스를 연결하는 데 사용되는 표준화된 프로토콜이며, A2A는 에이전트 간 협업을 위한 표준화된 프로토콜입니다."

</Task>

<Quality Checks>
EVERY claim is grounded in the provided Source material.
각 URL은 Source list에서 ONLY 한 번만 나타납니다.
introduction: # for Website Search title, no structural elements, no sources section.
conclusion: ## for Conclusion/Summary title, only ONE structural element at most, add sources and url section.
Markdown format.
word count 또는 preamble 포함되지 않습니다.
</Quality Checks>

<Task>
3. Web Search Queries:

다음 Web Search Queries를 추가합니다.

1.  "MCP vs A2A"
2.  "AI 에이전트간 협업"
3.  "AI 모델과 에이전트간 연결"

</Task>

<Quality Checks>
EVERY claim is grounded in the provided Source material.
각 URL은 Source list에서 ONLY 한 번만 나타납니다.
introduction: # for Website Search title, no structural elements, no sources section.
conclusion: ## for Conclusion/Summary title, only ONE structural element at most, add sources and url section.
Markdown format.
word count 또는 preamble 포함되지 않습니다.
</Quality Checks>